# 识别计算跳跃收益

## 导入模块

In [11]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import feather
import math
from scipy import stats
from joblib import Parallel, delayed
from tqdm.notebook import tqdm

## 读入测试数据

In [2]:
price_1m = feather.read_dataframe('../data/2024/StockPriceK1m_20240102.feather')
price_1m['date'] = pd.to_datetime(price_1m['date'].astype(str))
price_1d = feather.read_dataframe('../data/StockPriceK1d_20240630.feather')
price_1d['date'] = pd.to_datetime(price_1d['date'])

In [3]:
print(len(price_1m.loc[:13920, 'issue'].unique()))
print(len(price_1m['issue'].unique()))

59
5096


## 定义跳跃统计量

In [36]:
def mu(p: float):
    return (2 ** (p / 2)) * math.gamma((p + 1) / 2) / np.sqrt(np.pi)

mu1 = mu(1)
mu6 = mu(6)

# def JS(ret, log_ret):
#     n = len(ret)
#     abs_log_ret = np.abs(log_ret)
#     prod_6 = (
#         abs_log_ret[:-5] * 
#         abs_log_ret[1:-4] *
#         abs_log_ret[2:-3] *
#         abs_log_ret[3:-2] *
#         abs_log_ret[4:-1] *
#         abs_log_ret[5:]
#     )
#     sum_prod_6 = prod_6.sum()
#     coef_Omega = (mu6 / 9) * ((n ** 3) * (mu1 ** -6) / (n - 5))
#     Omega_SwV = coef_Omega * sum_prod_6
    
#     SwV_N = 2 * (ret - log_ret).sum()
    
#     if (Omega_SwV == 0 or SwV_N == 0):
#         return np.nan

#     prod_2 = abs_log_ret[:-1] * abs_log_ret[1:]
#     sum_prod_2 = prod_2.sum()
#     coef_V = 1 / mu1
#     V_01 = coef_V * sum_prod_2

#     RV_N = (log_ret * log_ret).sum()
    
#     js = n * (V_01 / np.sqrt(Omega_SwV)) * (1 - RV_N / SwV_N)
#     return js

def pvalue(js: float):
    cdf = stats.norm.cdf(js, loc=0, scale=1)
    return 2 * min(cdf, 1 - cdf)

In [47]:
def JS(ret, log_ret):
    n_series, n_points = ret.shape
    print(n_series, n_points)
    
    abs_log_ret = np.abs(log_ret)
    
    window_size = 6
    windows = np.lib.stride_tricks.sliding_window_view(
        abs_log_ret, window_shape=window_size, axis=1
    )
    prod_6 = np.prod(windows, axis=-1)
    sum_prod_6 = np.sum(prod_6, axis=1)
    
    coef_Omega = (mu6 / 9) * ((n_points ** 3) * (mu1 ** -6) / (n_points - 5))
    Omega_SwV = coef_Omega * sum_prod_6
    
    SwV_N = 2 * np.sum(ret - log_ret, axis=1)
    
    window_size_2 = 2
    windows_2 = np.lib.stride_tricks.sliding_window_view(
        abs_log_ret, window_shape=window_size_2, axis=1
    )
    prod_2 = np.prod(windows_2, axis=-1)
    sum_prod_2 = np.sum(prod_2, axis=1)
    
    coef_V = 1 / mu1
    V_01 = coef_V * sum_prod_2
    
    RV_N = np.sum(log_ret ** 2, axis=1)
    
    valid_mask = (Omega_SwV != 0) & (SwV_N != 0)
    js = np.full(n_series, np.nan)
    
    if np.any(valid_mask):
        valid_idx = np.where(valid_mask)[0]
        js[valid_idx] = n_points * (V_01[valid_idx] / np.sqrt(Omega_SwV[valid_idx])) * (1 - RV_N[valid_idx] / SwV_N[valid_idx])
    
    return js[0] if n_series == 1 else js

In [51]:
issue = '000001'
date = '2024-01-02'
date_next = '2024-01-03'
prc = price_1m.copy()
start_price = prc.loc[(price_1m['issue'] == issue) & (price_1m['time'] % 500 == 0), 'open'].to_numpy()
end_price = prc.loc[(price_1m['issue'] == issue) & (price_1m['time'] % 500 == 400), 'close'].to_numpy()

idx_d1 = (price_1d['issue'] == issue) &(price_1d['date'] == date)
idx_d2 = (price_1d['issue'] == issue) &(price_1d['date'] == date_next)
start_price = np.append(start_price, price_1d.loc[idx_d1, 'close'].to_numpy())
end_price = np.append(end_price, price_1d.loc[idx_d2, 'open'].to_numpy())

ret = (end_price - start_price) / start_price
log_ret = np.log(1 + ret)
n = len(ret)

med = np.median(ret)
log_med = np.median(log_ret)
ret_mat = np.tile(ret, (n, 1))
log_ret_mat = np.tile(log_ret, (n, 1))

array([[-0.00532481,  0.00214133, -0.00213675, ...,  0.        ,
        -0.0010846 , -0.00217155],
       [-0.00532481,  0.00214133, -0.00213675, ...,  0.        ,
        -0.0010846 , -0.00217155],
       [-0.00532481,  0.00214133, -0.00213675, ...,  0.        ,
        -0.0010846 , -0.00217155],
       ...,
       [-0.00532481,  0.00214133, -0.00213675, ...,  0.        ,
        -0.0010846 , -0.00217155],
       [-0.00532481,  0.00214133, -0.00213675, ...,  0.        ,
        -0.0010846 , -0.00217155],
       [-0.00532481,  0.00214133, -0.00213675, ...,  0.        ,
        -0.0010846 , -0.00217155]], shape=(49, 49))

## 识别跳跃, 计算收益

In [6]:
def jump_identify(ret, log_ret):
    n = len(ret)
    jump = np.full(n, False, dtype=bool)
    med = np.median(ret)
    log_med = np.median(log_ret)
    ret_c = ret.copy()
    log_ret_c = log_ret.copy()
    js0 = JS(ret_c, log_ret_c)
    p = pvalue(js0)
    
    while (p < 0.05):
        js = np.zeros(n)
        for i in range(n):
            r = ret_c.copy()
            lr = log_ret_c.copy()
            r[i] = med
            lr[i] = log_med
            js[i] = JS(r, lr)
        js_diff = np.abs(js0) - np.abs(js)
        idx_max = np.argmax(js_diff)
        jump[idx_max] = True
        ret_c[idx_max] = med
        log_ret_c[idx_max] = log_med
        js0 = JS(ret_c, log_ret_c)
        p = pvalue(js0)

    return np.any(jump), log_ret[jump].sum()

In [33]:
def jump_identify_price(price_1m, price_1d, date, date_next):
    # issues = price_1m['issue'].unique()
    # issue = issues[0]
    issue = price_1m.iloc[0]['issue']
    prc = price_1m.copy()
    start_price = prc.loc[price_1m['time'] % 500 == 0, 'open'].to_numpy()
    end_price = prc.loc[price_1m['time'] % 500 == 400, 'close'].to_numpy()
    
    idx_d1 = (price_1d['issue'] == issue) &(price_1d['date'] == date)
    idx_d2 = (price_1d['issue'] == issue) &(price_1d['date'] == date_next)
    start_price = np.append(start_price, price_1d.loc[idx_d1, 'close'].to_numpy())
    end_price = np.append(end_price, price_1d.loc[idx_d2, 'open'].to_numpy())
    
    ret = (end_price - start_price) / start_price
    log_ret = np.log(1 + ret)

    flag_jump, ret_jump = jump_identify(ret, log_ret)
    df_jump = pd.DataFrame({'issue': [issue], 'jump': [flag_jump], 'ret_jump': [ret_jump]})
    return df_jump

def jump_identify_parallel(price_1m, price_1d, date, date_next):
    groups = list(price_1m.groupby('issue'))
    total_groups = len(groups)
    
    results = Parallel(n_jobs=-1, backend='loky')(
        delayed(jump_identify_price)(group[1], price_1d, '2024-01-02', '2024-01-03')
        for group in tqdm(groups, total=total_groups, desc="Processing stocks")
    )
    
    return pd.concat(results)

In [34]:
%%time
df_jump = jump_identify_parallel(price_1m.loc[:13920], price_1d, '2024-01-02', '2024-01-03')
df_jump

Processing stocks:   0%|          | 0/59 [00:00<?, ?it/s]

CPU times: total: 34.8 s
Wall time: 54.7 s


,issue,jump,ret_jump
0,000001,True,-0.005339
0,000002,True,-0.009625
0,000004,True,0.008658
0,000005,False,0.000000
0,000006,False,0.000000
0,000007,False,0.000000
0,000008,False,0.000000
0,000009,True,0.001708
0,000010,False,0.000000
0,000011,True,-0.009060


In [35]:
%%time
df_jump = price_1m.loc[:13920].groupby('issue')[['issue', 'time', 'open', 'close']].apply(
    jump_identify_price,
    price_1d=price_1d,
    date='2024-01-02',
    date_next='2024-01-03'
)
df_jump

CPU times: total: 1min 51s
Wall time: 1min 57s


,,issue,jump,ret_jump
issue,,,,
000001,0,000001,True,-0.005339
000002,0,000002,True,-0.009625
000004,0,000004,True,0.008658
000005,0,000005,False,0.000000
000006,0,000006,False,0.000000
000007,0,000007,False,0.000000
000008,0,000008,False,0.000000
000009,0,000009,True,0.001708
000010,0,000010,False,0.000000
